In [1]:
import geopandas as gpd 
from pathlib import Path
import xarray as xr
import rioxarray as rxr

In [ ]:
from utils.read_files import read_from_control, make_default_path
# Store the name of the 'active' file in a variable
controlFile = 'control_EastRiver.txt'
# basin name and outlet gauge id
root_path = Path(read_from_control(controlFile, "root_path"))
basin = read_from_control(controlFile, "domain_name")

In [20]:
# open the geopotential data and take the temporal average
gph_ds = xr.open_dataset(root_path / "ERA5_surface_geopotential.nc").mean(dim="valid_time")['z']
# add the crs to the geopotential data
gph_ds = gph_ds.rio.write_crs("epsg:4326")

In [21]:
# open the basin shapefile
basin_gdf = gpd.read_file(root_path / f"domain_{basin}" / "shapefiles" / "catchment" / f"{basin}.shp")

In [22]:
# clip box the geopotential data to the basin
gph_ds_clipped = gph_ds.rio.clip_box(basin_gdf.total_bounds[0], basin_gdf.total_bounds[1], basin_gdf.total_bounds[2], basin_gdf.total_bounds[3])

In [23]:
gravity = 9.80665 # m/s^2

# calculate the geopotential height
gph_height = gph_ds_clipped / gravity

# add units
gph_height.attrs['units'] = 'm'

In [24]:
# save the geopotential height data
gph_height.to_netcdf(root_path / f"domain_{basin}" / "forcing" / "0_geopotential" / f"ERA5_surface_geopotential_height_{basin}.nc")